# Keras MF experimentation

This notebook experiments with building MF models in Keras on TensorFlow

In [1]:
import sys
import logging
_h = logging.StreamHandler(sys.stdout)
_lk_h = logging.getLogger('lenskit')
_lk_h.addHandler(_h)
_lk_h.setLevel(logging.INFO)

In [2]:
from lkdemo.datasets import ml20m

C:\Users\michaelekstrand\Anaconda3\envs\lk-demo\lib\site-packages\fastparquet\dataframe.py:5: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import CategoricalIndex, RangeIndex, Index, MultiIndex


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras as k

In [5]:
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender
from lenskit.algorithms.basic import Bias
from lenskit.algorithms.als import BiasedMF
from lenskit.algorithms.funksvd import FunkSVD
from lenskit import batch

In [6]:
ratings = ml20m.ratings

In [7]:
uidx = pd.Index(ratings['user'].unique())
iidx = pd.Index(ratings['item'].unique())
ratings['uno'] = uidx.get_indexer(ratings['user']).astype('i4')
ratings['ino'] = iidx.get_indexer(ratings['item']).astype('i4')
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   user       int32  
 1   item       int32  
 2   rating     float64
 3   timestamp  int32  
 4   uno        int32  
 5   ino        int32  
dtypes: float64(1), int32(5)
memory usage: 534.1 MB


In [8]:
n_users = len(uidx)
n_users

138493

In [9]:
n_items = len(iidx)
n_items

26744

In [10]:
train, test = next(xf.sample_users(ratings, 1, 10000, xf.SampleN(5)))

sampling 138493 users into 1 partitions (n=10000)


In [11]:
bias = Bias()
bias.fit(train)

building bias model for 19950263 ratings
global mean: 3.525
computed means for 26743 items
computed means for 138493 users


In [12]:
bias_preds = batch.predict(bias, test)
bias_preds['error'] = bias_preds['rating'] - bias_preds['prediction']
np.sqrt(np.mean(np.square(bias_preds['error'])))

using model store NoopModelStore
generating 50000 predictions for 10000 users


0.900496214297442

In [13]:
als = BiasedMF(25)
als.fit(train)

[ 0ms] fitting bias model
building bias model for 19950263 ratings
global mean: 3.525
computed means for 26743 items
computed means for 138493 users
[9.69s] normalizing 138493x26743 matrix (19950263 nnz)
[12.15s] training biased MF model with ALS for 25 features
[17.88s] finished epoch 0 (|ΔP|=474.305, |ΔQ|=180.702)
[20.71s] finished epoch 1 (|ΔP|=159.977, |ΔQ|=74.806)
[23.55s] finished epoch 2 (|ΔP|=108.074, |ΔQ|=54.944)
[26.57s] finished epoch 3 (|ΔP|=62.815, |ΔQ|=34.773)
[29.68s] finished epoch 4 (|ΔP|=42.396, |ΔQ|=24.184)
[32.72s] finished epoch 5 (|ΔP|=29.860, |ΔQ|=17.590)
[35.70s] finished epoch 6 (|ΔP|=20.781, |ΔQ|=13.112)
[38.71s] finished epoch 7 (|ΔP|=15.175, |ΔQ|=10.090)
[41.77s] finished epoch 8 (|ΔP|=11.492, |ΔQ|=7.980)
[44.75s] finished epoch 9 (|ΔP|=9.003, |ΔQ|=6.512)
[47.75s] finished epoch 10 (|ΔP|=7.294, |ΔQ|=5.478)
[50.88s] finished epoch 11 (|ΔP|=6.055, |ΔQ|=4.710)
[53.89s] finished epoch 12 (|ΔP|=5.102, |ΔQ|=4.105)
[56.88s] finished epoch 13 (|ΔP|=4.345, |ΔQ|=3.610

In [14]:
als_preds = batch.predict(als, test)
als_preds['error'] = als_preds['rating'] - als_preds['prediction']
np.sqrt(np.mean(np.square(als_preds['error'])))

using model store NoopModelStore
generating 50000 predictions for 10000 users


0.852457501634329

    funk = FunkSVD(25)
    funk.fit(train)

    funk_preds = batch.predict(funk, test)
    funk_preds['error'] = funk_preds['rating'] - funk_preds['prediction']
    np.sqrt(np.mean(np.square(funk_preds['error'])))

In [15]:
gbias = train['rating'].mean()
ntrs = train.assign(nrating = ratings['rating'] - gbias)
ibias = ntrs.groupby('item')['nrating'].mean().rename('i_bias')
ntrs = ntrs.join(ibias, on='item')
ntrs['nrating'] -= ntrs['i_bias'].fillna(0)
ubias = ntrs.groupby('user')['nrating'].mean().rename('u_bias')
ntrs = ntrs.join(ubias, on='user')
ntrs['nrating'] -= ntrs['u_bias']
ntrs.head()

,user,item,rating,timestamp,uno,ino,nrating,i_bias,u_bias
0,1,2,3.5,1112486027,0,0,0.174863,-0.313905,0.113771
1,1,29,3.5,1112484676,0,1,-0.566214,0.427172,0.113771
2,1,32,3.5,1112484819,0,2,-0.512570,0.373528,0.113771
3,1,47,3.5,1112484727,0,3,-0.667298,0.528257,0.113771
4,1,50,3.5,1112484580,0,4,-0.947923,0.808881,0.113771


## Regularized MF for explicit ratings

We're going to build up regularized MF for explicit ratings, based on [this tutorial](https://towardsdatascience.com/building-a-book-recommendation-system-using-keras-1fba34180699) and [this example code](https://github.com/chinchi-hsu/KerasCollaborativeFiltering):

In [16]:
features = 25

In [17]:
k.backend.set_floatx('float64')

In [31]:
graph = tf.Graph()
graph.seed = 42

First, the user layers:

In [32]:
with graph.as_default():
    u_input = k.Input(shape=(1,), dtype='int32', name='user')
    u_reg = k.regularizers.l2(0.02)
    u_embed = k.layers.Embedding(input_dim=n_users, output_dim=features, input_length=1,
                                 activity_regularizer=u_reg,
                                 embeddings_initializer='random_normal',
                                 name='user-embed')(u_input)
    u_flat = k.layers.Flatten(name='user-vector')(u_embed)

And the item layers:

In [33]:
with graph.as_default():
    i_input = k.Input(shape=(1,), dtype='int32', name='item')
    i_reg = k.regularizers.l2(0.02)
    i_embed = k.layers.Embedding(input_dim=n_items, output_dim=features, input_length=1,
                                 activity_regularizer=i_reg,
                                 embeddings_initializer='random_normal',
                                 name='item-embed')(i_input)
    i_flat = k.layers.Flatten(name='item-vector')(i_embed)

And put it together:

In [34]:
with graph.as_default():
    prod = k.layers.Dot(name='score', axes=1)([u_flat, i_flat])
    model = k.Model([u_input, i_input], prod, name='classic-mf')
    model.summary()

Model: "classic-mf"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
item (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
user-embed (Embedding)          (None, 1, 25)        3462325     user[0][0]                       
__________________________________________________________________________________________________
item-embed (Embedding)          (None, 1, 25)        668600      item[0][0]                       
_________________________________________________________________________________________

In [35]:
with graph.as_default():
    model.compile('adam', 'mean_squared_error', metrics=['mae'])

In [39]:
with graph.as_default():
    mfit = model.fit([ntrs.uno, ntrs.ino], ntrs.nrating, epochs=5, batch_size=1024*8)

Train on 19950263 samples
Epoch 1/5
19950263/19950263 [==============================] - 68s 3us/sample - loss: 0.6532 - mae: 0.6020
Epoch 2/5
19950263/19950263 [==============================] - 68s 3us/sample - loss: 0.6336 - mae: 0.5876
Epoch 3/5
19950263/19950263 [==============================] - 68s 3us/sample - loss: 0.6155 - mae: 0.5744
Epoch 4/5
19950263/19950263 [==============================] - 68s 3us/sample - loss: 0.6009 - mae: 0.5637
Epoch 5/5
19950263/19950263 [==============================] - 68s 3us/sample - loss: 0.5893 - mae: 0.5549


In [45]:
with graph.as_default():
    preds = model.predict([test.uno, test.ino])
preds = test.assign(pred=preds)
preds.head()

,user,item,rating,timestamp,uno,ino,pred
639,7,1894,3.0,1011208487,6,487,0.250061
543,7,11,4.0,1011207889,6,386,0.126278
581,7,806,1.0,1011206295,6,449,-0.808468
578,7,750,5.0,1011206659,6,448,-0.652010
769,7,3864,3.0,1011207079,6,588,0.174732


In [46]:
preds['pred'].describe()

count    50000.000000
mean         0.050540
std          0.339911
min         -2.228172
25%         -0.145796
50%          0.053081
75%          0.253099
max          2.793369
Name: pred, dtype: float64

In [47]:
preds = preds.join(ubias, on='user')
preds = preds.join(ibias, on='item')
preds['pred'] += gbias
preds['pred'] += preds.u_bias
preds['pred'] += preds.i_bias
preds['bpred'] = gbias
preds['bpred'] += preds.u_bias
preds['bpred'] += preds.i_bias
preds.head()

,user,item,rating,timestamp,uno,ino,pred,u_bias,i_bias,bpred
639,7,1894,3.0,1011208487,6,487,2.965026,-0.10908,-0.701225,2.714965
543,7,11,4.0,1011207889,6,386,3.685107,-0.10908,0.142639,3.558830
581,7,806,1.0,1011206295,6,449,2.242880,-0.10908,-0.364843,3.051347
578,7,750,5.0,1011206659,6,448,3.486772,-0.10908,0.722591,4.138782
769,7,3864,3.0,1011207079,6,588,2.566389,-0.10908,-1.024534,2.391657


In [48]:
preds['error'] = preds['rating'] - preds['pred']
np.sqrt(np.mean(np.square(preds['error'])))

0.8359782872463672

In [49]:
preds['berror'] = preds['rating'] - preds['bpred']
np.sqrt(np.mean(np.square(preds['berror'])))

0.9004962923324781